In [ ]:
%%capture
!pip install selenium

In [ ]:
%%capture
!pip install webdriver-manager

In [ ]:
# Importing All the Important Libraries 
import numpy as np
import pandas as pd
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

In [ ]:
Dates=pd.read_excel(r"./Dates.xlsx")

In [ ]:
# Cross Check Extracting Dates 
Dates['week']=np.where(Dates['Month'].isin(["Jan","May"]),Dates['week']+1,Dates['week'])
Dates.head(10)

In [ ]:
# Website which we will be using 
website='https://vegetablemarketprice.com/market/noida/today'

In [ ]:
# For getting into each Date
Data_list=[]
driver = webdriver.Chrome(options=webdriver.ChromeOptions() ) # opening the Browser
sleep(1)
driver.maximize_window() # Maximizing the windows
driver.get(website)     # Loading the Website
sleep(1)
for index in Dates.index:
    Desired_month= str(Dates['Month'][index]+ ' '+ str(Dates['year'][index]))
    week_row=Dates['week'][index]
    Day_cell=Dates['DayofWeek'][index]
    driver.find_element(By.XPATH, "/html/body/div[2]/div[2]/div[1]/div[2]/div[2]/div/div[1]").click()  #Calendar dropdown
    sleep(1)
    while driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[2]").text != Desired_month :
        driver.find_element(By.XPATH,"/html/body/div[4]/div[2]/div[1]/table/thead/tr[1]/th[1]").click()
        sleep(0.1)
    driver.find_element(By.XPATH,f'/html/body/div[4]/div[2]/div[1]/table/tbody/tr[{week_row}]/td[{Day_cell}]').click() # Date Select 
    try:
        myElem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'IdOfMyElement')))
        print("Page is ready!")
    except TimeoutException:
        print("Loading took too much time!")
    elements =driver.find_elements(By.CLASS_NAME,"todayVetableTableRows")
    sleep(1)
    for element in elements:
        id=element.get_attribute('id')
        Vegetable=driver.find_element(By.XPATH,'//*[@id="{}"]/td[2]'.format(str(id)))
        Wholesale=driver.find_element(By.XPATH,'//*[@id="{}"]/td[3]'.format(str(id)))
        Retail=   driver.find_element(By.XPATH,'//*[@id="{}"]/td[4]'.format(str(id)))
        Shopping= driver.find_element(By.XPATH,'//*[@id="{}"]/td[5]'.format(str(id)))
        Units=    driver.find_element(By.XPATH,'//*[@id="{}"]/td[6]'.format(str(id)))
        Date=     Dates['Date'][index]
        Data_list.append({'Date': Date,
            'Vegetable':Vegetable.text,
              'Wholesale Price':Wholesale.text,
              'Retail Price':Retail.text,
              'Shopping Mall Price':Shopping.text,
              'Units':Units.text})
        sleep(0.01)
print("Scrapping Done")

In [ ]:
df=pd.DataFrame(Data_list)

In [ ]:
# Checking How many Days Data We got 
df

In [ ]:
# Data Cleaning And Pre Processing and Converting it into Different 
df["Wholesale Price"]=df["Wholesale Price"].str.replace('₹','').astype(np.int64)
df["Retail Price"]=df["Retail Price"].str.replace('₹','')
df["Shopping Mall Price"]=df["Shopping Mall Price"].str.replace('₹','')
df["Retail Price"]=(df["Retail Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Retail Price"].str.split('-',expand=True)[0].astype(np.int64) )//2
df["Shopping Mall Price"]=(df["Shopping Mall Price"].str.split('-',expand=True)[1].astype(np.int64) +df["Shopping Mall Price"].str.split('-',expand=True)[0].astype(np.int64) )//2

In [ ]:
# Cross Checking did we get proper data or not 
df

In [ ]:
# Change File Name Accoring to Choice
df.to_csv('NoidaData1.csv')